In [33]:
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

import b_grapher.color as c

In [34]:
xls = pd.ExcelFile(
    # "./data/Student and Faculty Commons Survey_February 16, 2022_09.17.xlsx"
    "./data/Student and Faculty Commons Survey_February 16, 2022_09.17.xlsx"
)

data = pd.read_excel( xls, "Student and Faculty Commons Sur" )

useable_data = data.drop( [0, 1] )
useable_data = useable_data.drop( data.columns[0:17], axis=1 )

intersections = data[ ["intersection", "columnName"] ]

useable_data.fillna('No Response', inplace=True)    
intersections.fillna("blank", inplace=True)    


useable_data.head

/Users/brianmasse/opt/miniconda3/envs/CSC630/lib/python3.9/site-packages/pandas/core/frame.py:5176: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<bound method NDFrame.head of                  Q2            Q4    Q4_6_TEXT            Q8  \
2       No Response   No Response  No Response   No Response   
3             White   No Response  No Response   No Response   
4   Native American        Female  No Response   No Response   
5             White    Non-Binary  No Response  Heterosexual   
6       No Response   No Response  No Response   No Response   
7       White,Asian          Male  No Response  Heterosexual   
8       No Response   No Response  No Response   No Response   
9             Asian  Gender Fluid  No Response      Bisexual   
10            Asian        Female  No Response      Bisexual   
11            Asian        Female  No Response      Bisexual   
12      No Response   No Response  No Response   No Response   
13      White,Asian          Male  No Response  Heterosexual   
14      No Response   No Response  No Response   No Response   

                   Q10   Q10_6_TEXT                       Q9    Q9_8_TEXT

In [35]:
def return_slice_of_elements( element, arr ):
    array = arr.copy()
    start = array.index(element)
    array.reverse()
    end = len(array) - 1 - array.index(element)
    return ( start, end )

In [36]:
def render_graph(question, figure):
    traces = []
    cooresponding_intersections = []

    options = len(useable_data[question])
    values = np.full( options, 2 )

    buttons = []

    # This will put all the traces into a list
    for i, intersection in enumerate(intersections["columnName"]):
        if not intersection == "blank":
            fig_data = figure(question, intersection, values, i )
            traces += (fig_data)
            cooresponding_intersections = list(cooresponding_intersections) + list(np.full(len( fig_data ), str(intersection)).astype(str))


    # This add the buttons and intersections to all these traces 
    for i, intersection in enumerate(intersections["columnName"]):
        if not intersection == "blank":
            rows = intersections["columnName"] == intersection
            name = intersections.loc[ rows, "intersection" ].item()

            visibility = np.full( len(traces), False )

            slice = return_slice_of_elements( intersection, cooresponding_intersections) 
            visibility[slice[0]:slice[1] + 1] = True
            cooresponding_intersections.reverse()

            buttons.append(dict(label=str(name),
                        method="update",
                        args=[{"visible":visibility}] ))

    fig = go.Figure(data=traces)

    fig.update_layout(
        updatemenus=[
            dict(
                direction="down",
                active=0,
                x=0,
                y=1,
                buttons=list(buttons)
            )
        ])    

    return fig

In [37]:
def color_map( intersections ):
    returning = {}

    arr = useable_data[intersections]

    color1 = c.color( 252, 190, 154 )
    color2 = c.color( 245, 66, 120 )

    length = len( np.unique(arr) )
    
    i = 1
    for intersection in arr:
        if not intersection in returning:
            color = color1.return_color_between(color2, ( i / length )).return_string_in("RGB")
            returning[ intersection ] = color
            i += 1
    return returning

In [38]:
def render_sundial(question, intersection, values, i):
    return list(px.sunburst(useable_data, 
        path=[question, intersection], 
        values=values,
        color=intersection,
        color_discrete_map = color_map(intersection)
        
    ).update_traces(visible=True if i==0 else False).data)

def render_bar(question, intersection, values, i):
    return list(px.bar(useable_data, 
        x=question, 
        y=values, 
        color=intersection,
        color_discrete_map = color_map(intersection)
        
    ).update_traces(visible=True if i==0 else False).data)

In [39]:
def font(size):
    return dict(
        color="aliceblue",
        family="Monoid",
        size=size
    )

layout = dict (
    title =  dict(
        font=font(15),
        text="This is a title",
        x = 0.5,
    ),
    legend= dict(
        bgcolor="rgb(30, 30, 40)",
        bordercolor="rgb(20, 20, 30)",
        borderwidth=7,
        font= font(10),
        orientation="h",
        title = dict(
            font=font(15),
            # text="Legend Title",
        ),
    ),
    width = 800,
    height = 500,
    font=font(10),

    paper_bgcolor="rgb(30, 30, 40)",
    plot_bgcolor="rgb(20, 20, 30)",

    yaxis=dict(
        gridcolor="rgb(10, 10,15)",
        gridwidth=3,
        tickangle=90
    ),
    xaxis=dict(
        showgrid=False,
        tickangle=0
    ),


    grid=dict(
        columns=1,
        rows=1,
    ),

    bargap=0.2


)

In [40]:
graph = render_graph("Q14", render_bar)
graph.update_layout(layout)